# **Combinar Conjuntos de Datos con Merge y Join**

## Tabla de contenidos
1. [Álgebra Relacional](#0)
2. [Categorías de Uniones](#2)
3. [Especificación de la clave de combinación](#4)
4. [Especificación de aritmética de conjuntos para las uniones](#6)
5. [Nombres de columnas superpuestos: el parámetro `suffixes`](#8)

Una característica que ofrece Pandas son sus métodos de unión (Join) y combinación (Merge) en memoria de alto rendimiento.
Estos dos operadores son heredados de la manipulación e interacción de datos con motores de bases de datos.
El operador principal para esto es la función `pd.merge`, y se verán algunos ejemplos de cómo esto puede funcionar en la práctica.

Para mayor comodidad, se va a definir una clase `desplegar()` para mostrar los resultados de una manera visualmente atractiva.

In [ ]:
import pandas as pd

class desplegar(object):
    """ Muestra una representación con HTML de múltiples objetos """

    plantilla = """<div style="float: left; padding: 10px;">
                <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
                </div>"""

    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.plantilla.format(a, eval(a)._repr_html_())
                        for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 1. Álgebra Relacional <a id="0"></a>

El comportamiento implementado en `pd.merge()` es un subconjunto de lo que se conoce como **álgebra relacional**, que es un conjunto formal de reglas para manipular datos relacionales y constituye la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
La fuerza del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los componentes básicos de operaciones más complicadas en cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado de manera eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función `pd.merge()` y el método `join()` que está relacionado con los objetos del tipo `Series` y `Dataframe`. Todo esto permiten vincular de manera eficiente datos de diferentes fuentes.



## 2. Categorías de Uniones <a id="2"></a>

La función `pd.merge()` implementa varios tipos de uniones: las uniones **uno a uno**, **muchos a uno** y **muchos a muchos**.
Se accede a los tres tipos de uniones a través de una llamada idéntica a la interfaz `pd.merge()`; el tipo de unión realizada depende de la forma de los datos de entrada.
Aquí se mostrará ejemplos simples de los tres tipos de fusiones.

### Uniones uno a uno

La unión uno a uno es el tipo más simple de expresión de combinación, que en muchos aspectos es muy similar a la concatenación por columnas.
Como ejemplo concreto, se considerará los siguientes dos *dataframes* que contienen información sobre varios empleados de una empresa.

In [ ]:
df1 = pd.DataFrame({'empleado': ['Roberto', 'Javier', 'Luisa', 'Susana'],
                    'grupo': ['Contador', 'Ingeniero', 'Ingeniero', 'RRHH']})
df2 = pd.DataFrame({'empleado': ['Luisa', 'Roberto', 'Javier', 'Susana'],
                    'fecha_contratacion': [2004, 2008, 2012, 2014]})
desplegar('df1', 'df2')

Para combinar esta información en un solo *dataframe*, se puede usar la función `pd.merge()`.

In [ ]:
df3 = pd.merge(df1, df2)
df3

La función `pd.merge()` reconoce que cada *dataframe*  tiene una columna de `empleado` y se une automáticamente usando esta columna como clave.
El resultado de la fusión es un nuevo *dataframe* que combina la información de las dos entradas.
Tener en cuenta que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna `empleado` difiere entre `df1` y `df2`, y la función `pd.merge()` realiza el pareado de for correcta.
Además, tener en cuenta que la fusión en general descarta el índice, excepto en el caso especial de fusiones por índice a través de los parámetros `left_index` y `right_index`.

### Uniones de muchos a uno

Las uniones de muchos a uno son uniones en las que una de las dos columnas clave contiene entradas duplicadas. 
Para el caso de muchos a uno, el *dataframe* resultante conservará esas entradas duplicadas según corresponda.
Considerar el siguiente ejemplo de una combinación de muchos a uno.


In [ ]:
df4 = pd.DataFrame({'grupo': ['Contador', 'Ingeniero', 'RRHH'],
                    'supervisor': ['Carla', 'Guido', 'Sergio']})
desplegar('df3', 'df4', 'pd.merge(df3, df4)')

El *dataframe* resultante tiene una columna adicional con la información del `supervisor`, donde la información se repite en una o más ubicaciones según lo requieran las entradas.

### Uniones de muchos a muchos

Las uniones de muchos a muchos son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna clave en la matriz izquierda y derecha contiene duplicados, entonces el resultado es una combinación de muchos a muchos.
Esto será quizás más claro con un ejemplo concreto.
Considere lo siguiente, se tiene un *dataframe* que muestra una o más habilidades asociadas con un grupo en particular.
Al realizar una unión de muchos a muchos, se puede recuperar las habilidades asociadas con cualquier persona individual.

In [ ]:
df5 = pd.DataFrame({'grupo': ['Contador', 'Contador',
                              'Ingeniero', 'Ingeniero', 'RRHH', 'RRHH'],
                    'habilidades': ['matematicas', 'planillas', 'codificacion', 'linux',
                               'planillas', 'organizacion']})
desplegar('df1', 'df5', "pd.merge(df1, df5)")

Estos tres tipos de uniones se pueden usar con otras herramientas de Pandas para implementar una amplia gama de funciones.
Pero en la práctica, los conjuntos de datos rara vez están tan limpios como con el que se está trabajando aquí.
A continuación se consideraran algunas de las opciones proporcionadas por `pd.merge()` que permiten ajustar cómo funcionan las operaciones de combinación.

## 3. Especificación de la clave de combinación <a id="4"></a>

Ya se ha visto el comportamiento predeterminado de `pd.merge()`, el cuál busca uno o más nombres de columna coincidentes entre las dos entradas y lo usa como clave.
Sin embargo, a menudo los nombres de las columnas no coincidirán tan bien, y `pd.merge()` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

De manera más simple, se puede especificar explícitamente el nombre de la columna clave usando el parámetro `on`, que toma un nombre de columna o una lista de nombres de columna.

In [ ]:
desplegar('df1', 'df2', "pd.merge(df1, df2, on='empleado')")

Esta opción solo funciona si tanto el *dataframe* izquierdo como el derecho tienen el nombre de columna especificado.

### Los parámetros `left_on` y `right_on`

En ocasiones, es posible que se desee fusionar dos conjuntos de datos con diferentes nombres de columna; por ejemplo, se puede tener un conjunto de datos en el que el nombre del empleado esté etiquetado como `nombre` en lugar de `empleado`.
En este caso, se puede usar los parámetros `left_on` y `right_on` para especificar los dos nombres de columna.

In [ ]:
df3 = pd.DataFrame({'nombre': ['Roberto', 'Javier', 'Luisa', 'Susana'],
                    'sueldo': [70000, 80000, 120000, 90000]})
desplegar('df1', 'df3', 'pd.merge(df1, df3, left_on="empleado", right_on="nombre")')

El resultado tiene una columna redundante que se puede descartar si se desea, por ejemplo, usando el método `drop()` de los *dataframes*.

In [ ]:
pd.merge(df1, df3, left_on="empleado", right_on="nombre").drop('nombre', axis=1)

### Los parámetros `left_index` y `right_index`

Podría suceder que en lugar de combinarse en una columna, se desea realizar una combinación en un índice.
Por ejemplo, los datos se podrían verse así.

In [ ]:
df1a = df1.set_index('empleado')
df2a = df2.set_index('empleado')
desplegar('df1a', 'df2a')

Se puede usar el índice como la clave para combinar especificando los parámetros `left_index` y/o `right_index` en `pd.merge()`.

In [ ]:
desplegar('df1a', 'df2a', 'pd.merge(df1a, df2a, left_index=True, right_index=True)')

Para mayor comodidad, los *dataframes* tienen implementado el método `.join()`, que realiza una combinación que por defecto se une a traves de los índices.

In [ ]:
desplegar('df1a', 'df2a', 'df1a.join(df2a)')

Si desea mezclar índices y columnas, se puede combinar `left_index` con `right_on` o `left_on` con `right_index` para obtener el comportamiento que se desea.

In [ ]:
desplegar('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='nombre')")

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para obtener más información al respecto, consultar la [sección "Fusionar, unir y concatenar"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## 4. Especificación de aritmética de conjuntos para las uniones <a id="6"></a>

En todos los ejemplos anteriores, se ha pasado por alto una consideración importante al realizar una combinación, el cuál es el tipo de aritmética de conjuntos utilizada en la combinación.
Esto surge cuando aparece un valor en una columna clave pero no en la otra. Considerar este ejemplo.

In [ ]:
df6 = pd.DataFrame({'nombre': ['Pedro', 'Pablo', 'Maria'],
                    'comida': ['pescado', 'legumbres', 'pan']},
                   columns=['nombre', 'comida'])
df7 = pd.DataFrame({'nombre': ['Maria', 'Jose'],
                    'bebida': ['vino', 'cerveza']},
                   columns=['nombre', 'bebida'])
desplegar('df6', 'df7', 'pd.merge(df6, df7)')

Aquí se han fusionado dos conjuntos de datos que solo tienen una sola entrada de **"nombre"** en común: Maria.
Por defecto, el resultado contiene la **intersección** de los dos conjuntos de entradas; esto es lo que se conoce como **unión interna**.
Se puede especificar esto explícitamente usando el parámetro `how`, que por defecto es `'inner'`.

In [ ]:
pd.merge(df6, df7, how='inner')

Otras opciones para el parámetro `how` son `'outer'`, `'left'` y `'right'`.
Una **unión externa** o `'outer'` devuelve una combinación sobre la unión de las columnas de entrada y completa todos los valores faltantes con NA.

In [ ]:
desplegar('df6', 'df7', "pd.merge(df6, df7, how='outer')")

La **unión izquierda** o `'left'` y la **unión derecha** o `'right'` devuelven combinaciones sobre las entradas izquierda y derecha, respectivamente.
Por ejemplo.

In [ ]:
desplegar('df6', 'df7', "pd.merge(df6, df7, how='left')")

Las filas de salida ahora corresponden a las entradas en la entrada izquierda. 
Usando `how='right'` funciona de manera similar pero conservando las entradas de la entrada derecha.

In [ ]:
desplegar('df6', 'df7', "pd.merge(df6, df7, how='right')")

Todas estas opciones se pueden aplicar directamente a cualquiera de los tipos de combinación anteriores.

## 5. Nombres de columnas superpuestos: el parámetro `suffixes` <a id="8"></a>

Finalmente, se puede terminar en un caso en el que los dos *dataframes* de entrada tengan nombres de columnas en conflicto.
Considerar el siguiente ejemplo.

In [ ]:
df8 = pd.DataFrame({'nombre': ['Roberto', 'Javier', 'Luisa', 'Susana'],
                    'ranking': [1, 2, 3, 4]})
df9 = pd.DataFrame({'nombre': ['Roberto', 'Javier', 'Luisa', 'Susana'],
                    'ranking': [3, 1, 4, 2]})
desplegar('df8', 'df9', 'pd.merge(df8, df9, on="nombre")')

Debido a que la salida tendría dos nombres de columna en conflicto, la función de combinación agrega automáticamente un sufijo `_x` o `_y` para que las columnas de salida sean únicas.
Si estos valores predeterminados no son apropiados, es posible especificar un sufijo personalizado usando el parámetro `suffixes`.

In [ ]:
desplegar('df8', 'df9', 'pd.merge(df8, df9, on="nombre", suffixes=["_L", "_R"])')

Estos sufijos funcionan en cualquiera de los posibles patrones de combinación y también funcionan si hay varias columnas superpuestas.

Consulte también la [documentación "Fusionar, unir y concatenar" de Pandas](http://pandas.pydata.org/pandas-docs/stable/merging.html) para obtener más información sobre estos temas.